In [1]:
import os
import sys
import time
import pennylane as qml
from pennylane import numpy as np

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
from measurement_memory import *
from pauli_grouping import *
from Experiment_Molecular import single_qubit_basis_rotation, givens_rotation, phase_rotation
from Hamiltonians.utils import read_Hamiltonian_simple0, read_Hamiltonian_FG0

In [2]:
def To_pennylane(Hg:list):
    obs = []
    coeffs = []
    for i in range(len(Hg)):
        for j,p in enumerate(str_to_Pauli(Hg[i][0])):
            obs.append(p)
            coeffs.append(Hg[i][1][j])
    qmlH = qml.Hamiltonian(coeffs, obs)
    return qmlH

## H2

In [3]:
# Define ansatz
N = 4
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#### 1. Original H

In [143]:
H = read_Hamiltonian0("molecular", mol="H2", grouping_type="GC")
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(0.08419299)

#### 2. QWC evaluation

In [14]:
H = read_Hamiltonian0("molecular", mol="H2", grouping_type="QWC")

In [15]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        E += evaluate_eigenstate(circuit_qwc(x, M), h)
    return E

evaluate(params)

-0.25768313576632806

#### 3. GC evaluation

In [16]:
H = read_Hamiltonian0("molecular", mol="H2", grouping_type="GC")
Hg, basis = basis_transformation(H)

In [17]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(T, Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(x, T, Q), h)
    return E

evaluate(params)

-0.2566501063981185

#### 4. FG evaluation

In [4]:
# Reference H from OpenFermion
H = read_Hamiltonian_simple0("molecular", mol="H2", grouping_type="ref")
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

print(circuit_fg(params))

-0.07004117467781192


In [5]:
H, G, P = read_Hamiltonian_FG0('H2')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    phase_rotation(p)
    givens_rotation(g)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

print(evaluate(params))

-0.07004117467781223


In [6]:
P

[[(1.0, 1.0, 1.0, 1.0)],
 [(1.0, 1.0, 1.0, 1.0)],
 [(1.0, 1.0, 1.0, 1.0)],
 [(1.0, 1.0, 1.0, 1.0)]]

# H4

In [22]:
# Define ansatz
N = 8
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#### 1. Original H

In [5]:
H = read_Hamiltonian0("molecular", mol="H4", grouping_type="GC")
qmlH = To_pennylane(H)

In [17]:
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(1.24332087)

#### 2. QWC evaluation

In [16]:
H = read_Hamiltonian_simple0("molecular", mol="H4", grouping_type="QWC")

In [9]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        #print(M)
        E += evaluate_eigenstate(circuit_qwc(x, M), h)
    return E

evaluate(params)

0.9144130703743264

#### 3. GC evaluation

In [10]:
from Experiment_Molecular import measurement_rotation

H = read_Hamiltonian0("molecular", mol="H4", grouping_type="GC")
Hg, basis = basis_transformation(H)

In [11]:
dev = qml.device("lightning.qubit", wires=N, shots=999999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(T, Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(x, T, Q), h)
    return E

evaluate(params)

0.9145625886790075

#### 4. FG evaluation

In [89]:
params = 2*np.pi*np.random.rand(N*2)

In [90]:
# Reference H from OpenFermion
H = read_Hamiltonian_simple0("molecular", mol="H4", grouping_type="ref")
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)
ref = circuit_fg(params)
print(ref)

0.5327812681381434


In [91]:
H, G, P = read_Hamiltonian_FG0('H4')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    phase_rotation(p)
    givens_rotation(g)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

0.5292027043326835


In [92]:
print((fge-ref)/ref)

-0.006716759802696515


## LiH

In [79]:
# Define ansatz
N = 12
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

In [80]:
# Reference H from OpenFermion
H = read_Hamiltonian_simple0("molecular", mol="LiH", grouping_type="ref")
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

ref = circuit_fg(params)
print(ref)

-4.787972006252776


In [106]:
H, G, P = read_Hamiltonian_FG0('LiH')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    phase_rotation(p)
    givens_rotation(g)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

-3.219588847202436


In [82]:
print((fge-ref)/ref)

-8.579092570330081e-05


In [108]:
G[1]

[(-1.5707972852524996, 10, 11),
 (0.014383618379312731, 10, 11),
 (1.7250308549325668, 8, 9),
 (3.141592653589793, 9, 10),
 (0.014382264705381537, 10, 11),
 (3.141276394241187, 7, 8),
 (1.4159590286485353, 8, 9),
 (-1.565475104173961, 10, 11),
 (1.5707966372481263, 6, 7),
 (1.5708467389339635, 7, 8),
 (1.5708432036624695, 8, 9),
 (3.141592653589793, 9, 10),
 (-1.565691857667739, 10, 11),
 (1.5707963267945753, 5, 6),
 (1.5707963267948954, 6, 7),
 (1.570796326794887, 7, 8),
 (1.5707963267948966, 8, 9),
 (2.499256603862893, 9, 10),
 (-1.5709695645424264, 10, 11),
 (1.5707963267948966, 4, 5),
 (1.570796326794906, 5, 6),
 (1.5707963267948966, 6, 7),
 (1.5707963267945741, 7, 8),
 (1.5706230890473667, 8, 9),
 (2.499256603862495, 9, 10),
 (-0.0001303634898947352, 10, 11),
 (1.5707963267950313, 3, 4),
 (1.5707963267948974, 4, 5),
 (1.5707963267949006, 5, 6),
 (1.5707963267948966, 6, 7),
 (0.9939558626035448, 7, 8),
 (1.570678329668451, 8, 9),
 (1.570862317105958, 9, 10),
 (-1.10219912205397, 10

## H2O

In [102]:
# Define ansatz
N = 14
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

In [103]:
# Reference H from OpenFermion
H = read_Hamiltonian_simple0("molecular", mol="H2O", grouping_type="ref")
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

ref = circuit_fg(params)
print(ref)

-40.187948067139054


In [104]:
H, G, P = read_Hamiltonian_FG0('H2O')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    phase_rotation(p)
    givens_rotation(g)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

-40.187964891440046


In [105]:
print((fge-ref)/ref)

4.186404581856583e-07
